In [1]:
import os, sys, json, hashlib, traceback, uuid
from datetime import datetime, timezone

BANNER = "🧱🔗 Basic Python Blockchain — Reliable • Beautiful • Checkpointed"
ART_DIR = "blockchain_artifacts"
CHK_FILE = os.path.join(ART_DIR, "checkpoints.jsonl")
CHAIN_FILE = os.path.join(ART_DIR, "blockchain.json")
LOG_FILE = os.path.join(ART_DIR, "run_log.jsonl")

def print_banner():
    print("\n" + "═" * 88)
    print(BANNER.center(88))
    print("═" * 88 + "\n")

def p_step(msg):
    print(f"🟦 STEP: {msg}")

def p_ok(msg):
    print(f"✅ OK: {msg}")

def p_info(msg):
    print(f"ℹ️  INFO: {msg}")

def p_warn(msg):
    print(f"⚠️  WARN: {msg}")

def p_err(msg):
    print(f"🛑 ERROR: {msg}")

def handle_error(e, context=""):
    etype = type(e).__name__
    emsg = str(e)
    tb = traceback.format_exc(limit=6)
    print("\n" + "✳️" * 30)
    p_err("A failure occurred")
    print(f"🧩 Context: {context}")
    print(f"👾 Exception: {etype}")
    print(f"📝 Message: {emsg}")
    print("🧵 Traceback (last frames):")
    print(tb)
    print("🌟 Resolution strategies (5-Star):")
    print("1) Re-run the previous cell to ensure all definitions are loaded and the state is clean.")
    print("2) Verify Python version ≥ 3.8 and standard library availability in the Kaggle CPU runtime.")
    print("3) If files are corrupted, delete the 'blockchain_artifacts' folder and re-run to regenerate safely.")
    print("4) Ensure there is enough disk space and permissions for writing artifacts in the working directory.")
    print("5) Restart the kernel if the notebook state seems inconsistent, then execute cells top-to-bottom.")
    print("✳️" * 30 + "\n")

class SafeIO:
    def __init__(self, artifacts_dir):
        self.artifacts_dir = artifacts_dir
        self.ensure_dir()

    def ensure_dir(self):
        os.makedirs(self.artifacts_dir, exist_ok=True)

    def safe_write_json(self, path, obj):
        tmp = f"{path}.tmp-{uuid.uuid4().hex}"
        with open(tmp, "w", encoding="utf-8") as f:
            json.dump(obj, f, ensure_ascii=False, indent=2, sort_keys=True)
        os.replace(tmp, path)

    def append_jsonl(self, path, obj):
        with open(path, "a", encoding="utf-8") as f:
            f.write(json.dumps(obj, ensure_ascii=False, sort_keys=True) + "\n")

class CheckpointManager:
    def __init__(self, io, chk_path):
        self.io = io
        self.chk_path = chk_path

    def checkpoint(self, stage, payload=None):
        record = {
            "id": uuid.uuid4().hex,
            "stage": stage,
            "payload": payload or {},
            "ts": datetime.now(timezone.utc).isoformat()
        }
        self.io.append_jsonl(self.chk_path, record)
        p_ok(f"Checkpoint saved 🧭 — {stage}")

def print_env(io):
    print_banner()
    p_step("Detecting environment and preparing directories")
    io.ensure_dir()
    cwd = os.getcwd()
    files = sorted(os.listdir("."))
    is_kaggle = bool(os.environ.get("KAGGLE_KERNEL_RUN_TYPE")) or os.path.exists("/kaggle")
    p_info(f"Working directory: {cwd}")
    p_info(f"Detected Kaggle runtime: {is_kaggle}")
    p_info(f"Python: {sys.version.split()}")
    p_info(f"Artifacts directory: {io.artifacts_dir}")
    p_info(f"Existing files here: {(', '.join(files))[:400] + ('...' if len(', '.join(files)) > 400 else '')}")

try:
    io = SafeIO(ART_DIR)
    ckpt = CheckpointManager(io, CHK_FILE)
    print_env(io)
    ckpt.checkpoint("environment_ready", {"python": sys.version, "artifacts_dir": ART_DIR})
    p_ok("Setup complete. Ready for blockchain definitions 🧰")
except Exception as e:
    handle_error(e, "Cell 1 — environment/setup")


════════════════════════════════════════════════════════════════════════════════════════
            🧱🔗 Basic Python Blockchain — Reliable • Beautiful • Checkpointed            
════════════════════════════════════════════════════════════════════════════════════════

🟦 STEP: Detecting environment and preparing directories
ℹ️  INFO: Working directory: /kaggle/working
ℹ️  INFO: Detected Kaggle runtime: True
ℹ️  INFO: Python: ['3.11.13', '(main,', 'Jun', '4', '2025,', '08:57:29)', '[GCC', '11.4.0]']
ℹ️  INFO: Artifacts directory: blockchain_artifacts
ℹ️  INFO: Existing files here: .virtual_documents, blockchain_artifacts
✅ OK: Checkpoint saved 🧭 — environment_ready
✅ OK: Setup complete. Ready for blockchain definitions 🧰


In [2]:
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any

def iso_utc_now():
    return datetime.now(timezone.utc).isoformat()

def deterministic_digest(payload: Dict[str, Any]) -> str:
    blob = json.dumps(payload, separators=(",", ":"), sort_keys=True, ensure_ascii=False).encode("utf-8")
    return hashlib.sha256(blob).hexdigest()

@dataclass
class Block:
    index: int
    timestamp: str
    transactions: List[Dict[str, Any]]
    previous_hash: str
    hash: str = field(init=False)

    def __post_init__(self):
        content = {
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash
        }
        self.hash = deterministic_digest(content)

    def to_dict(self) -> Dict[str, Any]:
        return {
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash,
            "hash": self.hash
        }

try:
    p_step("Defining Block with deterministic SHA-256 hashing")
    test_block = Block(index=0, timestamp=iso_utc_now(), transactions=[{"note":"genesis"}], previous_hash="0"*64)
    p_ok(f"Block model ready 🧩 — sample hash: {test_block.hash[:16]}...")
    ckpt.checkpoint("block_model_ready", {"sample_block_hash": test_block.hash})
except Exception as e:
    handle_error(e, "Cell 2 — block model")

🟦 STEP: Defining Block with deterministic SHA-256 hashing
✅ OK: Block model ready 🧩 — sample hash: 0f8a1c8380f6305b...
✅ OK: Checkpoint saved 🧭 — block_model_ready


In [3]:
import copy

class Blockchain:
    def __init__(self):
        self.chain: List[Block] = []
        self.create_genesis_block()

    def create_genesis_block(self):
        if self.chain:
            return
        b = Block(
            index=0,
            timestamp=iso_utc_now(),
            transactions=[{"system":"genesis", "message":"👋 Welcome to the chain"}],
            previous_hash="0"*64
        )
        self.chain.append(b)

    def last_block(self) -> Block:
        return self.chain[-1]

    def add_block(self, transactions: List[Dict[str, Any]]) -> Block:
        lb = self.last_block()
        nb = Block(
            index=lb.index + 1,
            timestamp=iso_utc_now(),
            transactions=transactions,
            previous_hash=lb.hash
        )
        self.chain.append(nb)
        return nb

    def to_json(self) -> Dict[str, Any]:
        return {"length": len(self.chain), "chain": [b.to_dict() for b in self.chain]}

    @staticmethod
    def from_json(obj: Dict[str, Any]) -> "Blockchain":
        bc = Blockchain()
        bc.chain = []
        for b in obj.get("chain", []):
            rebuilt = Block(
                index=b["index"],
                timestamp=b["timestamp"],
                transactions=b["transactions"],
                previous_hash=b["previous_hash"]
            )
            bc.chain.append(rebuilt)
        if not bc.chain:
            bc.create_genesis_block()
        return bc

    def is_valid(self) -> Dict[str, Any]:
        if not self.chain:
            return {"ok": False, "reason": "empty_chain", "index": None}
        for i in range(1, len(self.chain)):
            cur = self.chain[i]
            prev = self.chain[i-1]
            if cur.previous_hash != prev.hash:
                return {"ok": False, "reason": "broken_link", "index": i}
            check = Block(
                index=cur.index,
                timestamp=cur.timestamp,
                transactions=cur.transactions,
                previous_hash=cur.previous_hash
            )
            if check.hash != cur.hash:
                return {"ok": False, "reason": "hash_mismatch", "index": i}
        return {"ok": True, "reason": None, "index": None}

    def save(self, path: str):
        obj = self.to_json()
        io.safe_write_json(path, obj)

    @staticmethod
    def load(path: str) -> "Blockchain":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        return Blockchain.from_json(obj)

try:
    p_step("Defining Blockchain container with validation and persistence")
    _bc_probe = Blockchain()
    _status = _bc_probe.is_valid()
    p_ok(f"Blockchain core ready 🏗️ — genesis status: {json.dumps(_status)}")
    ckpt.checkpoint("blockchain_core_ready", {"genesis_hash": _bc_probe.last_block().hash})
except Exception as e:
    handle_error(e, "Cell 3 — blockchain core")

🟦 STEP: Defining Blockchain container with validation and persistence
✅ OK: Blockchain core ready 🏗️ — genesis status: {"ok": true, "reason": null, "index": null}
✅ OK: Checkpoint saved 🧭 — blockchain_core_ready


In [4]:
try:
    p_step("Creating or resuming blockchain and adding sample transactions")
    if os.path.exists(CHAIN_FILE):
        p_warn("Existing chain detected — resuming from artifact")
        bc = Blockchain.load(CHAIN_FILE)
    else:
        bc = Blockchain()

    samples = [
        [{"from":"alice","to":"bob","amount":10},{"note":"coffee"}],
        [{"from":"bob","to":"carol","amount":5},{"note":"snack"}],
        [{"from":"carol","to":"dave","amount":2},{"note":"tip"}]
    ]

    for tset in samples:
        nb = bc.add_block(tset)
        p_ok(f"Added block #{nb.index} 🔗 hash: {nb.hash[:16]}… prev: {nb.previous_hash[:16]}…")

    bc.save(CHAIN_FILE)
    ckpt.checkpoint("blocks_appended", {"length": len(bc.chain), "last_hash": bc.last_block().hash})
    p_ok(f"Chain saved to {CHAIN_FILE} 💾")
except Exception as e:
    handle_error(e, "Cell 4 — build sample chain")

🟦 STEP: Creating or resuming blockchain and adding sample transactions
✅ OK: Added block #1 🔗 hash: 5605a419edbb2021… prev: 42d400fac6fa4f56…
✅ OK: Added block #2 🔗 hash: f401899320fe61e7… prev: 5605a419edbb2021…
✅ OK: Added block #3 🔗 hash: 90cf9634dc7653e7… prev: f401899320fe61e7…
✅ OK: Checkpoint saved 🧭 — blocks_appended
✅ OK: Chain saved to blockchain_artifacts/blockchain.json 💾


In [5]:
try:
    p_step("Validating current blockchain")
    bc = Blockchain.load(CHAIN_FILE)
    val = bc.is_valid()
    if val["ok"]:
        p_ok("Chain integrity verified ✅")
    else:
        p_err(f"Validation failed at index {val['index']} — {val['reason']}")

    p_step("Demonstrating tamper detection on a copy")
    bad = Blockchain.load(CHAIN_FILE)
    if len(bad.chain) > 1:
        bad.chain[1].transactions.append({"from":"eve","to":"mallory","amount":999})
        tv = bad.is_valid()
        if tv["ok"]:
            p_warn("Unexpected: tampered copy still valid")
        else:
            p_ok(f"Tampering detected as expected 🧨 index={tv['index']} reason={tv['reason']}")
    else:
        p_info("No second block to tamper with; add more blocks to see detection")

    ckpt.checkpoint("validation_done", {"valid": val["ok"]})
except Exception as e:
    handle_error(e, "Cell 5 — validate and tamper demo")

🟦 STEP: Validating current blockchain
✅ OK: Chain integrity verified ✅
🟦 STEP: Demonstrating tamper detection on a copy
✅ OK: Tampering detected as expected 🧨 index=1 reason=hash_mismatch
✅ OK: Checkpoint saved 🧭 — validation_done


In [6]:
def print_block(b: Block):
    print("─" * 88)
    print(f"🧱 Block #{b.index}")
    print(f"⏱️  Timestamp: {b.timestamp}")
    print(f"🔗 Previous: {b.previous_hash}")
    print(f"🔒 Hash:     {b.hash}")
    print("🗃️  Transactions:")
    if not b.transactions:
        print("   • (none)")
    else:
        for i, t in enumerate(b.transactions, 1):
            print(f"   • {i}: {json.dumps(t, ensure_ascii=False, sort_keys=True)}")

try:
    p_step("Loading and printing the full blockchain")
    bc = Blockchain.load(CHAIN_FILE)
    for blk in bc.chain:
        print_block(blk)
    io.safe_write_json(CHAIN_FILE, bc.to_json())
    ckpt.checkpoint("printed_and_exported", {"export_path": CHAIN_FILE, "blocks": len(bc.chain)})
    p_ok("Printed chain and re-exported JSON 📤")
    p_info(f"Artifacts: {ART_DIR} — {', '.join(sorted(os.listdir(ART_DIR)))}")
except Exception as e:
    handle_error(e, "Cell 6 — pretty print/export")

🟦 STEP: Loading and printing the full blockchain
────────────────────────────────────────────────────────────────────────────────────────
🧱 Block #0
⏱️  Timestamp: 2025-09-03T18:12:18.635154+00:00
🔗 Previous: 0000000000000000000000000000000000000000000000000000000000000000
🔒 Hash:     42d400fac6fa4f56903505f25dcb117160f671dc9b47f087bb475bec96582904
🗃️  Transactions:
   • 1: {"message": "👋 Welcome to the chain", "system": "genesis"}
────────────────────────────────────────────────────────────────────────────────────────
🧱 Block #1
⏱️  Timestamp: 2025-09-03T18:12:18.635235+00:00
🔗 Previous: 42d400fac6fa4f56903505f25dcb117160f671dc9b47f087bb475bec96582904
🔒 Hash:     5605a419edbb2021f496beb5cd13d19ef638a7cb3c68ea4a906c5e6b1c008f8d
🗃️  Transactions:
   • 1: {"amount": 10, "from": "alice", "to": "bob"}
   • 2: {"note": "coffee"}
────────────────────────────────────────────────────────────────────────────────────────
🧱 Block #2
⏱️  Timestamp: 2025-09-03T18:12:18.635279+00:00
🔗 Previous: 5605